# ITC 2007


In [1]:
# Installing dependencies
%pip install -q amplpy pandas numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Google Colab & Kaggle integration
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["gurobi"],  # modules to install
    license_uuid="acf35a17-6898-4d26-ab01-a6cc68a46872",  # license to use
)

Licensed to Bundle #6876.7352 expiring 20250415: Decision Support Methods ("M?todos de Apoio ? Decis?o"), Jo?o Pedro Pedroso, University of Porto.


## ITC2007 Model

Based on the Second International Timetabling Competition: Examination timetabling track:

1. Sets and Parameters

    Set `EXAM`: set of exams 

    Parameters:
    - `sizeExam {EXAM}`: size of exam e
    - `durationExam {EXAM}`: duration of exam e
    - `exammFL {EXAM}`: boolean that is 1 iff exam e is subject to Front-Load penalties, 0 otherwise

    SET `STUDENT`: set of students

    Parameters:
    - `sizeStudent {STUDENT}`: size of students within a set of students that have the same exams

    SET `STUDENT_EXAMS`: set that contains for each exam the students enrolled in it

    SET `DURATION`: set of durations used in exams

    Parameters:
    - `duration {EXAM, DURATION}`: boolean that is 1 iff exam e has "duration type" d

    SET `PERIOD`: set of periods

    Parameters:
    - `durationPeriod {PERIOD}`: duraion of period p
    - `periodFL {PERIOD}`: boolean that is 1 iff period p is subject to the FrontLoad penalties
    - `weightPeriod {PERIOD}`: weight that specifies the penalty for using period p
    - `same_day {PERIOD, PERIOD}`: boolean that is 1 iff periods p and q are in the same day

    SET `ROOM`: set of rooms

    Parameters:
    - `sizeRoom {ROOM}`: size of room r
    - `weightRoom {ROOM}`: weight that specifies the penalty for using room r

2. Period Related Hard Constraints

    SET `AFTER`: a set of pairs of exams
    For every pair (e1, e2) ∈ AFTER exam e1 must occur strictly after exam e2

    SET `COINCIDENCE`: a set of pairs of exams
    For every pair (e1, e2) ∈ COINCIDENCE exam e1 and e2 must occur in the same period

    SET `EXCLUSION`: a set of pairs of exams
    For every pair (e1, e2) ∈ EXCLUSION exam e1 and e2 must not occur in the same period


3. Room Related Hard Constraints

    SET `EXCLUSIVE`: a set of exams
    For every exam e ∈ EXCLUSIVE, if exam e is assigned to period p and room r then e must be the sole occupier, i.e. no other exam can be assigned to both p and r

4. Institutional Weights and Parameters

    Parameters:
    - `TWOINAROW`: weight for "two in a row"
    - `TWOINADAY`: weight for "two in a day"
    - `PERIODSPREAD`: weight for period spread (defaults to 1 as not currently specified in the input format)
    - `NONMIXEDDURATIONS`: weight for "No mixed duration"
    - `FRONTLOAD`: weight for the Front load penalty
    - `gPERIODSPREAD`: the period spread, the preferred minimal "gap" between exams for a student
    
5. Variables:

    5.1 Primary Decision Variables

    - `X_P {EXAM, PERIOD}`: boolean that is 1 if exam i is in period p otherwise 0
    - `X_R {EXAM, PERIOD}`: boolean that is 1 if exam i is in room r otherwise 0
    - `Y_P {EXAM, PERIOD}`: fraction of students for each room

    5.2 Secondary Variables

    - `C_2R {STUDENT} >= 0`: two-in-a-row penalty for each student s
    - `C_2D {STUDENT} >= 0`: two-in-a-day penalty for each student s
    - `C_PS {STUDENT} >= 0`: period spread penalty for each student s
    - `C_NMD >= 0`: no mixed duration penalty
    - `C_FL >= 0`: front-load penalty
    - `C_P >= 0`: soft period penalty
    - `C_R >= 0`: soft room penalty

6. Objective:
    
    - `Total_Penalty`: minimise total cost of penalties 

7. Constraints:

    7.1 Hard Constraints:

    - `(10) and (11)`: every exam is allocated to at least one room, and to at least one period
    - `(12)`: room capacities are always respected 
    - `(13)`: period durations are respected
    - `(14)`: in any period, any student is taking at most one exam
    - `(15), (16) and (17)`: hard period constraints
    - `(18)`: hard room constraints
    - `(29)`:
    - `(30)`:

    7.2 Soft Constraints:
    - `(19)`: two in a row
    - `(20)`: two in a day
    - `(21)`: period spread
    - `(22), (23), (24) and (25)`: non-mixed durations 
    - `(26)`: front load
    - `(27)`: soft period penalties
    - `(28)`: soft room penalties

This track is then modeled as follows:

In [3]:
%%ampl_eval
reset;
set EXAM;

param sizeExam {EXAM} >= 0;
param durationExam {EXAM} > 0;
param examFL {EXAM} binary;

set STUDENT;

param sizeStudent {STUDENT} > 0;

set STUDENT_EXAMS {STUDENT} within EXAM;

set DURATION;

param duration {EXAM, DURATION} binary;

set PERIOD;

param durationPeriod {PERIOD} > 0;
param periodFL {PERIOD} binary;
param weightPeriod {PERIOD} >= 0;

param same_day {PERIOD, PERIOD} binary;

set ROOM;

param sizeRoom {ROOM} > 0;
param weightRoom {ROOM} >= 0;

set AFTER within {EXAM, EXAM};
set COINCIDENCE within {EXAM, EXAM};
set EXCLUSION within {EXAM, EXAM};

set EXCLUSIVE within EXAM;

# Weighting parameters
param TWOINAROW;
param TWOINADAY;
param PERIODSPREAD := 1;
param NONMIXEDDURATIONS;
param FRONTLOAD;
param gPERIODSPREAD;

### Imports

In [4]:
import sys
import pandas as pd
import numpy as np
import re
from collections import defaultdict

### File paths


In [5]:
lines_file_path = 'exam_sets.txt'
input_file_path = 'datasets/exam_comp_set1.exam'
output_file_path = 'exam_sets.txt'
solution_file_path = 'solutions/solution.txt'

In [6]:
# Parameters
input_file_path = "datasets/exam_comp_set12.exam"
solution_file_path = "solutions/solution12.txt"


### File creation with information on input file

In [7]:
size = {}

# Extracting information from file
with open(input_file_path, 'r') as file:
    lines = file.readlines()  # Reading all lines at once for easier processing
    line_count = len(lines)

    i = 0
    while i < line_count:
        line = lines[i]

        # Using regex to capture bracket pair in order to extract to file
        matches = re.findall(r'\[(.*?):(.*?)\]', line)

        # Checking for size
        if matches:
            for match in matches:
                word, value = match[0].strip(), match[1].strip()
                size[word] = value
        else:
            if line.startswith('[') and line.endswith(']\n'):
                parameter_name = line[1:-2].strip()
                if parameter_name not in size:
                    # Counting lines until next parameter
                    count_lines = 0
                    j = i + 1
                    while j < line_count and not re.findall(r'\[(.*?)]', lines[j]):
                        count_lines += 1
                        j += 1
                    size[parameter_name] = count_lines
                    i = j - 1

        i += 1

# Writing to file
with open(output_file_path, 'w') as file:
    for param in size:
        file.write(f'{param} {size[param]}\n')

print("Set and size_of file created at:", output_file_path)


Set and size_of file created at: exam_sets.txt


### Functions to be used

In [8]:
# Function to find a specific line with the text
def find_lines(filename, text):
    try:
        with open(filename, 'r') as file:
            for line in file:
                if text in line:
                    parts = line.split()
                    if len(parts) > 1:
                        return int(parts[1]) # Returns the number where the text is in the file
        return None # Return None if the text is not found
    except FileNotFoundError:
        print(f"The file {filename} was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Function to find and check a specific line count for a given text
def check_lines(filename, text):
    line_count = find_lines(filename, text)
    if line_count is not None:
        print(f"The lines of {text} are {line_count}")
        return line_count
    else:
        print(f"{text} not found in the file.")
        sys.exit(1)

# Function to find a specific line by the first word
def find_line_by_first_word(filename, first_word):
    try:
        with open(filename, 'r') as file:
            for line in file:
                if line.startswith(first_word):
                    return line.strip().split(',')  # Return the full line as a string
        return None  # Return None if the line is not found
    except FileNotFoundError:
        print(f"The file {filename} was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

### Preprocessing

In [9]:
# Reading file
with open(input_file_path, 'r') as file:
    lines = file.readlines()

# Number of lines of each main cluster
exam_lines = check_lines(lines_file_path, "Exams")
period_lines = check_lines(lines_file_path, "Periods")
room_lines = check_lines(lines_file_path, "Rooms")
periodHard_lines = check_lines(lines_file_path, "PeriodHardConstraints")
roomHard_lines = check_lines(lines_file_path, "RoomHardConstraints")
weight_lines = check_lines(lines_file_path, "InstitutionalWeightings")

# Getting Front-Load values
line_data = find_line_by_first_word(input_file_path, "FRONTLOAD")
examFL = int(line_data[1].strip())
periodFL = int(line_data[2].strip())
wFL = int(line_data[3].strip())

The lines of Exams are 78
The lines of Periods are 12
The lines of Rooms are 50
The lines of PeriodHardConstraints are 9
The lines of RoomHardConstraints are 7
The lines of InstitutionalWeightings are 5


### Exam and Students Processing

In [10]:
exam_start_line = 1

# Initializing lists to store exam and student data
exam_data = []
student_exam = []
student_exams = {}
all_students = set()
i = 0

for index in range(exam_start_line, exam_start_line + exam_lines):
    if index < len(lines):
        line_data = [int(x.strip()) for x in lines[index].split(',')]

        # Extracting duration and student IDs while also creating size and checking for frontload
        exam = i
        duration = line_data[0]
        student_ids = line_data[1:]
        size = len(student_ids)
        frontload = 1 if size >= examFL else 0

        # Storing exam data
        exam_data.append((exam, size, duration, frontload))
        # Storing students for each Exam
        student_exam.append(student_ids)
        # Adding student IDs to the global set of all students
        all_students.update(student_ids)
        i += 1

exam_df = pd.DataFrame(exam_data, columns=["EXAM", "sizeExam", "durationExam", "examFL"],).set_index("EXAM")

# Creating exam labels and sorting all students
exam_labels = [index for index in range(0, len(exam_data))]
sorted_students = sorted(all_students)

# Initializing empty lists for each student in student_exams
student_exams = {student_id: [] for student_id in sorted_students}

# Filling the dictionary for each student
for student_exam, student_ids in enumerate(student_exam):
    for student_id in student_ids:
        student_exams[student_id].append(student_exam)

# Grouping students by their exact set of exams using defaultdict
exam_sets = defaultdict(list)
for student, exams in student_exams.items():
    exam_sets[frozenset(exams)].append(student)

# Dictionary to store the size of students represented
student_size = {}
student_exams.clear()

for exam_set, students in exam_sets.items():
    # Picking the first student as the representative
    representative = students[0]
    # Only the representative in student_exams
    student_exams[representative] = list(exam_set)
    # Recording the count of students represented
    student_size[representative] = len(students)

# Extracting unique durations
unique_durations = sorted({duration for _, _, duration, _ in exam_data})

# Creating labels for columns in the duration matrix
duration_labels = [d for d in unique_durations]

# Initializing a matrix with zeros for duration data (rows for exams, columns for unique durations)
duration_matrix = np.zeros((len(exam_data), len(unique_durations)), dtype=int)

# Filling the duration matrix
for exam_idx, (_, _, duration, _) in enumerate(exam_data):
    duration_col = unique_durations.index(duration)   # Finding row index for the current exam's duration
    duration_matrix[exam_idx, duration_col] = 1       # Marking duration as 1

duration_df = pd.DataFrame(duration_matrix, columns=duration_labels, index=[i for i in range(len(exam_data))])

### Period Processing

In [11]:
period_start_line = 2 + exam_lines

# Initializing lists to store period data
period_data = []
period_dates = []
unloaded = period_lines - periodFL
i = 0

for index in range(period_start_line, period_start_line + period_lines):
    if index < len(lines):
        line_data = lines[index].split(',')

        # Extracting date of exam, duration, penalty and checking for frontload
        period = i
        date = line_data[0].strip()
        time = int(line_data[2].strip())
        penalty = int(line_data[3].strip())
        frontload = 1 if i > unloaded else 0

        # Storing date of exam
        period_dates.append(date)
        # Storing the period data
        period_data.append((period, time, frontload, penalty))
        i += 1

period_df = pd.DataFrame(period_data, columns=["PERIOD", "durationPeriod", "periodFL", "weightPeriod"],).set_index("PERIOD")

# Creating period labels
period_labels = [index for index in range(0, len(period_dates))]

# Initializing a matrix with zeros (rows and columns for periods)
same_day_matrix = np.zeros((len(period_dates), len(period_dates)), dtype=int)

# Filling same_day matrix
for i in range(len(period_dates)):
    for j in range(len(period_dates)):
        if period_dates[i] == period_dates[j]:   # Checking to see if date of period i and j are same
            same_day_matrix[i, j] = 1            # If yes marking as 1

same_day_df = pd.DataFrame(same_day_matrix, columns=period_labels, index=period_labels)

### Room Processing

In [12]:
room_start_line = 3 + exam_lines + period_lines

# Initializing lists to store room data
room_data = []
i = 0

for index in range(room_start_line, room_start_line + room_lines):
    if index < len(lines):
        line_data = lines[index].split(',')

        # Extracting size and penalty
        room = i
        size = int(line_data[0].strip())
        penalty = int(line_data[1].strip())

        room_data.append((room, size, penalty))
        i += 1

room_df = pd.DataFrame(room_data, columns=["ROOM", "sizeRoom", "weightRoom"],).set_index("ROOM")

### Period Hard Constraints Processing

In [13]:
periodHard_start_line = 4 + exam_lines + period_lines + room_lines

# Initializing lists to store periodHard data
after_data = []
coincidence_data = []
exclusion_data = []

for index in range(periodHard_start_line, periodHard_start_line + periodHard_lines):
    if index < len(lines):
        line_data = lines[index].split(',')

        # Extracting exams and type of constraint
        fExam = int(line_data[0].strip())
        operation = line_data[1].strip()
        sExam = int(line_data[2].strip())

        if operation == "AFTER":
            after_data.append((fExam, sExam))
        elif operation == "EXAM_COINCIDENCE":
            coincidence_data.append((fExam, sExam))
        elif operation == "EXCLUSION":
            exclusion_data.append((fExam, sExam))

### Room Hard Constraints Processing

In [14]:
roomHard_start_line = 5 + exam_lines + period_lines + room_lines + periodHard_lines

# Initializing list to store roomHard penalties
roomHard_data = []

for index in range(roomHard_start_line, roomHard_start_line + roomHard_lines):
    if index < len(lines):
        line_data = lines[index].split(',')
        exam = int(line_data[0].strip())

        roomHard_data.append(exam)

### Institutional Weightings Processing

In [15]:
weight_start_line = 6 + exam_lines + period_lines + room_lines + periodHard_lines + roomHard_lines

# Initializing list to weights data
weight_data = []

for index in range(weight_start_line, weight_start_line + weight_lines):
    if index < len(lines):
        line_data = lines[index].strip().split(',')

        # Extracting parameter and weight
        key = line_data[0].strip()
        if key != "FRONTLOAD":
            weight_data.append((key, int(line_data[1].strip())))
        elif key == "FRONTLOAD":
            weight_data.append((key, int(line_data[3].strip())))

weight_dict = {key: value for key, value in weight_data}

### Loading Exam, student and duration data

In [16]:
# Sending data from "exam_df" to AMPL and initializing set "EXAM"
ampl.set_data(exam_df, "EXAM")
# Setting the keys of "STUDENT" set in AMPL using "student_exams"
ampl.getSet("STUDENT").setValues(student_exams.keys())
# Setting the values of parameter "sizeStudent" using "student_size"
ampl.getParameter("sizeStudent").setValues(student_size)
# Setting the values of parameter "enrolled" using "enrollment_df"
ampl.set['STUDENT_EXAMS'].setValues(student_exams)
# Sending data from "unique_durations" to AMPL and initializing set "DURATION"
ampl.getSet("DURATION").setValues(duration_labels)
# Setting the values of parameter "duration" using "duration_df"
ampl.getParameter("duration").setValues(duration_df)

### Loading Period data

In [17]:
# Sending data from "period_df" to AMPL and initializing set "PERIOD"
ampl.set_data(period_df, "PERIOD")
# Setting the values of parameter "same_day" using "same_day_df"
ampl.getParameter("same_day").setValues(same_day_df)

### Loading Room data

In [18]:
# Sending data from "room_df" to AMPL and initializing set "ROOM"
ampl.set_data(room_df, "ROOM")

### Loading Period Hard Constraints data

In [19]:
# Sending data from "after_data" to AMPL and intializing set "AFTER"
ampl.getSet("AFTER").setValues(after_data)
# Sending data from "coincidence_data" to AMPL and intializing set "COINCIDENCE"
ampl.getSet("COINCIDENCE").setValues(coincidence_data)
# Sending data from "exclusion_data" to AMPL and intializing set "EXCLUSION"
ampl.getSet("EXCLUSION").setValues(exclusion_data)

### Loading Room Hard Constraint data

In [20]:
# Sending data from "roomHard_data" to AMPL and intializing set "EXCLUSIVE"
ampl.getSet("EXCLUSIVE").setValues(roomHard_data)

### Loading Institutional Weightings data

In [21]:
ampl.param["TWOINAROW"] = weight_dict.get("TWOINAROW", 1)
ampl.param["TWOINADAY"] = weight_dict.get("TWOINADAY", 1)
ampl.param["NONMIXEDDURATIONS"] = weight_dict.get("NONMIXEDDURATIONS", 1)
ampl.param["FRONTLOAD"] = weight_dict.get("FRONTLOAD", 1)
ampl.param["gPERIODSPREAD"] = weight_dict.get("PERIODSPREAD", 1)

### Variables

In [22]:
%%ampl_eval
# Primary Variables
var X_P {EXAM, PERIOD} binary; 
var X_R {EXAM, ROOM} binary; 
var MI_R {EXAM, ROOM} >= 0, <= 1; 

# Secondary Variables for Penalties
var C_2R {STUDENT} >= 0; 
var C_2D {STUDENT} >= 0; 
var C_PS {STUDENT} >= 0; 
var C_NMD >= 0;         
var C_FL >= 0;          
var C_P >= 0;           
var C_R >= 0;           


### Objective

In [23]:
%%ampl_eval
# Objective Function

minimize Total_Penalty:
    sum {s in STUDENT} (TWOINAROW * C_2R[s] + TWOINADAY * C_2D[s] + PERIODSPREAD * C_PS[s]) * sizeStudent[s]
    + NONMIXEDDURATIONS * C_NMD
    + FRONTLOAD * C_FL
    + C_P + C_R;

### Hard Constraints

In [24]:
%%ampl_eval
# (10)
s.t. AssignToOneRoom {i in EXAM}:
    sum {r in ROOM} X_R[i, r] >= 1;
# (11)
s.t. AssignToOnePeriod {i in EXAM}:
    sum {p in PERIOD} X_P[i, p] >= 1;
# (12)
s.t. RoomCapacity {p in PERIOD, r in ROOM}:
    sum {i in EXAM} sizeExam[i] * X_P[i, p] * MI_R[i, r] <= sizeRoom[r];
# (13)
s.t. PeriodDuration {p in PERIOD, i in EXAM}:
    durationExam[i] * X_P[i, p] <= durationPeriod[p];
# (14)
s.t. NoStudentConflict {p in PERIOD, s in STUDENT}:
    sum {i in STUDENT_EXAMS[s]} X_P[i, p] <= 1;
# (15)
s.t. Precedence {(i, j) in AFTER, p in PERIOD, q in PERIOD: q >= p}:
    X_P[i, p] + X_P[j, q] <= 1;
# (16)
s.t. Coincidence {(i, j) in COINCIDENCE, p in PERIOD}:
    X_P[i, p] = X_P[j, p];
# (17)
s.t. Exclusion {(i, j) in EXCLUSION, p in PERIOD}:
    X_P[i, p] + X_P[j, p] <= 1;
# (18)
s.t. HardRoomConstraints {i in EXCLUSIVE, j in EXAM, p in PERIOD, r in ROOM: j != i}:
    (X_P[i, p] + X_R[i, r] + X_P[j, p] + X_R[j, r]) <= 3;
# (29)
s.t. MI_RUsage {i in EXAM, r in ROOM}:
    MI_R[i, r] <= X_R[i, r];
# (30)
s.t. MI_count {i in EXAM}:
    sum {r in ROOM} MI_R[i, r] == 1;


### Soft Constraints

In [25]:
%%ampl_eval
# (19)
s.t. TwoInARowPenalty {s in STUDENT}:
    C_2R[s] = sum {i in STUDENT_EXAMS[s], j in STUDENT_EXAMS[s], p in PERIOD, q in PERIOD: j != i && q = p + 1 && same_day[p, q] = 1}
                  X_P[i, p] * X_P[j, q];
# (20)
s.t. TwoInADayPenalty {s in STUDENT}:
    C_2D[s] = sum {i in STUDENT_EXAMS[s], j in STUDENT_EXAMS[s], p in PERIOD, q in PERIOD: j != i && q > p + 1 && same_day[p, q] = 1}
                  X_P[i, p] * X_P[j, q];
# (21)
s.t. PeriodSpreadPenalty {s in STUDENT}:
    C_PS[s] = sum {i in STUDENT_EXAMS[s], j in STUDENT_EXAMS[s], p in PERIOD, q in PERIOD: j != i && p < q && q <= p+gPERIODSPREAD}
                  X_P[i, p] * X_P[j, q];

# Auxiliary variables for C_NMD calculations
var U_D {DURATION, PERIOD, ROOM} binary;  # 1 if a duration type d is assigned to (p, r)
var C_NMD_pr {PERIOD, ROOM} >= 0;         # Non-negative penalty for period-room pair

# (22)
s.t. Define_U_D {d in DURATION, i in EXAM, p in PERIOD, r in ROOM: duration[i, d] = 1}:
    U_D[d, p, r] >= X_P[i, p] + X_R[i, r] - 1;
# (23)
s.t. NonMixedDurationsPenalty {p in PERIOD, r in ROOM}:
    1 + C_NMD_pr[p, r] >= sum {d in DURATION} U_D[d, p, r];
# (24)
# No additional code is required for this since its already handled by the variable declaration
# (25)
s.t. Total_NonMixedDurations_Penalty:
    C_NMD = sum {p in PERIOD, r in ROOM} C_NMD_pr[p, r];
# (26)
s.t. FrontLoadPenalty:
    C_FL = sum {i in EXAM, p in PERIOD} examFL[i] * periodFL[p] * X_P[i, p];
# (27)
s.t. SoftPeriodPenalty:
    C_P = sum {p in PERIOD, i in EXAM} weightPeriod[p] * X_P[i, p];
# (28)
s.t. SoftRoomPenalty:
    C_R = sum {r in ROOM, i in EXAM} weightRoom[r] * X_R[i, r];


# Solve with HiGHS

In [26]:
# Specifying the solver to use
ampl.option["solver"] = "gurobi"
# Setting time limit of 2 hours
ampl.option['gurobi_options'] = 'timelimit=7200  outlev=1'
# Solving
ampl.solve()
# Stopping if the model was not solved
assert ampl.get_value("solve_result") == "solved"
# Getting objective entity by AMPL name
totalpenalty = ampl.get_objective("Total_Penalty")
print("Objective is:", totalpenalty.value())

Gurobi 12.0.1: 

  lim:time = 7200


Set parameter LogToConsole to value 1
  tech:outlev = 1



AMPL MP initial flat model has 11590 variables (0 integer, 5973 binary);
Objectives: 1 linear; 
Constraints:  353924 linear; 1714 quadratic;

AMPL MP final model has 11590 variables (0 integer, 5973 binary);
Objectives: 1 linear; 
Constraints:  353924 linear; 1714 quadratic;


Set parameter InfUnbdInfo to value 1
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 24.04.2 LTS")

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads



Non-default parameters:
TimeLimit  7200


...

     0     0  103.28966    0  604 10888.0000  103.28966  99.1%     -  139s


     0     0  103.29195    0  601 10888.0000  103.29195  99.1%     -  140s


     0     0  103.30007    0  600 10888.0000  103.30007  99.1%     -  140s


     0     0  103.31620    0  599 10888.0000  103.31620  99.1%     -  140s


     0     0  103.32859    0  597 10888.0000  103.32859  99.1%     -  141s


In [ ]:
# Retrieving X_P, X_R and MI_R variables and converting to a Pandas DataFrame
X_P = ampl.get_variable("X_P").to_pandas()
X_R = ampl.get_variable("X_R").to_pandas()
MI_R = ampl.get_variable("MI_R").to_pandas()

# Renaming the indexes on the Pandas DataFrame
X_P.index.names = ["EXAM", "PERIOD"]
X_R.index.names = ["EXAM", "ROOM"]
MI_R.index.names = ["EXAM", "ROOM"]

# Getting penalty values from AMPL
C_2R = ampl.get_variable("C_2R").to_pandas()  # TwoInARowPenalty
C_2D = ampl.get_variable("C_2D").to_pandas()  # TwoInADayPenalty
C_PS = ampl.get_variable("C_PS").to_pandas()  # PeriodSpreadPenalty
C_NMD = ampl.get_variable("C_NMD").to_pandas()  # Total_NonMixedDurations_Penalty
C_FL = ampl.get_variable("C_FL").to_pandas()  # FrontLoadPenalty
C_P = ampl.get_variable("C_P").to_pandas()  # SoftPeriodPenalty
C_R = ampl.get_variable("C_R").to_pandas()  # SoftRoomPenalty

# Computing totals
total_C_2R = C_2R["C_2R.val"].sum()
total_C_2D = C_2D["C_2D.val"].sum()
total_C_PS = C_PS["C_PS.val"].sum()
total_C_NMD = C_NMD["C_NMD.val"].sum() 
total_C_FL = C_FL["C_FL.val"].sum() 
total_C_P = C_P["C_P.val"].sum()
total_C_R = C_R["C_R.val"].sum()

with open(solution_file_path, "w") as file:
    file.write(X_P.to_string() + "\n\n")
    file.write(X_R.to_string() + "\n\n")
    file.write(MI_R.to_string() + "\n\n")
    file.write(f"Two In A Row Penalty: {total_C_2R}\n")
    file.write(f"Two In A Day Penalty: {total_C_2D}\n")
    file.write(f"Period Spread Penalty: {total_C_PS}\n")
    file.write(f"Non-Mixed Durations Penalty: {total_C_NMD}\n")
    file.write(f"Front Load Penalty: {total_C_FL}\n")
    file.write(f"Soft Period Penalty: {total_C_P}\n")
    file.write(f"Soft Room Penalty: {total_C_R}\n")